In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo-preview")


Let's create a vector Database

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb, os

embedding = OpenAIEmbeddings()

raw_documents = DirectoryLoader("./docs",glob="**/*.txt", show_progress=False).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(raw_documents)

chroma_directory = "chromadb"
if os.path.exists(chroma_directory):
    db = Chroma(embedding_function=embedding, persist_directory=chroma_directory)
else:
    db = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory=chroma_directory)

In [4]:
from langchain.chains import RetrievalQA
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=db.as_retriever(), chain_type_kwargs={"prompt": prompt}
)

Define a tool to fetch crime record

In [5]:
from langchain.agents import Tool, AgentExecutor, ZeroShotAgent
import requests
import json

def fetch_crime_records(json_request: str):
    url = 'http://localhost:8080/'
    endpoint = 'search_crime_records'
    arguments = json.loads(json_request)
    location = arguments.get('location',None)
    crime_type = arguments.get('crime_type',None)
    r = requests.get(url + endpoint, params={'location': location, 'crime_type': crime_type})
    return r.content

fetch_crime_records_tool = Tool(
    name="Get Crime record",
    func=fetch_crime_records,
    description="""Fetches the crime records based on location or crime type or both using API endpoints,
    input should be a JSON string with location and crime_type as keys""",
    handle_tool_error=True
)





In [6]:
from langchain.chains import LLMChain
prompt = ZeroShotAgent.create_prompt(
    tools=[fetch_crime_records_tool],
    prefix="You are a helpful assistant that helps people get crime records. {instruction}\n\
            You have this chat history: {chat_history}\n\n",
    format_instructions="""use the following format:\n
    Question: the input question you must answer.
    Thought: you should always thing about what to do.
    Action: the action to take, should be one of the tools {tool_names}
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: Summarize the observation which is the final answer to the original input question
    """,
    suffix="Begin:\nQuestion: {input}\nThought: {agent_scratchpad}",
    input_variables=['input', 'instruction'],
)
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

In [7]:
agent = ZeroShotAgent(
    llm_chain = llm_chain,
    tools = [fetch_crime_records_tool],
    verbose=True
)


c:\Users\manoj\Documents\MANOJ\Github\Ask-API\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


In [8]:
from langchain.memory import ConversationTokenBufferMemory

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[fetch_crime_records_tool],
    handle_parsing_errors=True,
    memory=ConversationTokenBufferMemory(memory_key="chat_history", max_token_limit=500,llm=llm),
    max_iterations=5,
    verbose=True
)

In [9]:
question = "Get crime records based on a crime type Homicide?"

In [10]:
instruction_from_llm = qa_chain.invoke({"query": question})['result']
agent_executor.run({"input": question,"instruction": instruction_from_llm})

c:\Users\manoj\Documents\MANOJ\Github\Ask-API\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that helps people get crime records. To get crime records based on the crime type Homicide, you should utilize the Search Endpoint of the API framework, setting the parameters to the crime type of interest, which in this case is "Homicide." This will allow you to retrieve relevant crime records that match your specified criteria. Ensure to also include any other relevant parameters such as location or date if needed to refine your search.
            You have this chat history: 



Get Crime record: Fetches the crime records based on location or crime type or both using API endpoints,
    input should be a JSON string with location and crime_type as keys

use the following format:

    Question: the input question you must answer.
    Thought: you should always thing about what to do.
    Action: the action to take, should be one of the tools Get Crime record


'The data retrieved provides an extensive list of homicide crime records, including details such as Crime Record ID, Location, Suspects, Victims, Witnesses, and Description of the incident, along with the date and time each crime was reported. This information is crucial for analysis, investigation, or research purposes related to these specific cases.'